In [2]:
import warnings
warnings.filterwarnings('ignore')
import sys

import keras
import numpy as np
from keras_preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Activation, Conv2D, Dense, Dropout, Flatten, MaxPooling2D, BatchNormalization
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.utils import class_weight

Using TensorFlow backend.


In [5]:
datagen = ImageDataGenerator(rotation_range=30, 
                             width_shift_range=0.2,
                             height_shift_range=0.2,
                             horizontal_flip=True,
                             vertical_flip=True
                            )

if 'win32' in sys.platform:
    data_directory='F:\\Segmentation_Data\\'
    full_training_directory = data_directory + 'Labelled_imgs\\data_for_generator\\training\\'
    full_validation_directory = data_directory +'Labelled_imgs\\data_for_generator\\validation\\'

if 'darwin' in sys.platform:
    data_directory='/Volumes/Samsung_T5/Segmentation_Data'
    full_training_directory = data_directory + 'Labelled_imgs\\data_for_generator\\training\\'
    full_validation_directory = data_directory +'Labelled_imgs\\data_for_generator\\validation\\'


In [6]:
batch_size = 32
train_generator = datagen.flow_from_directory(full_training_directory, 
                                              target_size=(128, 128), 
                                              color_mode="rgb", 
                                              batch_size=batch_size)
validation_generator = datagen.flow_from_directory(full_validation_directory,
                                                   target_size=(128, 128), 
                                                   color_mode='rgb',
                                                   batch_size=batch_size, 
                                                   shuffle=False)
num_classes = len(train_generator.class_indices)
num_test_samples = 8524

class_weights = class_weight.compute_class_weight(
           'balanced',
            np.unique(train_generator.classes), 
            train_generator.classes)

Found 19890 images belonging to 7 classes.
Found 8524 images belonging to 7 classes.


In [4]:
model = Sequential()
model.add(Conv2D(20, (5,5), activation='relu', input_shape=(128,128,3)))
model.add(BatchNormalization(momentum=0.9))
model.add(MaxPooling2D())
model.add(Conv2D(50, (5,5), activation='relu'))
model.add(BatchNormalization(momentum=0.9))
model.add(MaxPooling2D())
model.add(Conv2D(100, (4,4), activation='relu'))
model.add(MaxPooling2D())
model.add(Conv2D(200, (4,4), activation='relu'))
model.add(MaxPooling2D())
model.add(Dropout(rate=0.25))
model.add(Flatten())
model.add(Dense(500))
model.add(Dropout(rate=0.25))
model.add(Dense(num_classes))
model.add(Activation('softmax'))


W0822 18:05:53.412897 13624 deprecation_wrapper.py:119] From C:\Users\acsch\Anaconda3\envs\Tensorflow-GPU\lib\site-packages\keras\backend\tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0822 18:05:53.439185 13624 deprecation_wrapper.py:119] From C:\Users\acsch\Anaconda3\envs\Tensorflow-GPU\lib\site-packages\keras\backend\tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0822 18:05:53.447649 13624 deprecation_wrapper.py:119] From C:\Users\acsch\Anaconda3\envs\Tensorflow-GPU\lib\site-packages\keras\backend\tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0822 18:05:53.471950 13624 deprecation_wrapper.py:119] From C:\Users\acsch\Anaconda3\envs\Tensorflow-GPU\lib\site-packages\keras\backend\tensorflow_backend.py:174: The name tf.get_default_session is deprecated. Please use tf.compat.v1.get_de

In [5]:
sgd = keras.optimizers.SGD(lr=0.01,
                          momentum=0.9,
                          decay=0.0001)

In [24]:

model.compile(optimizer=sgd, loss='categorical_crossentropy', metrics=['accuracy'])
model.fit_generator(train_generator,
                    verbose=1, 
                    steps_per_epoch=1000, 
                    epochs=1, 
                    validation_data=validation_generator,
                   validation_steps=300)

Epoch 1/1
1000/1000 [==============================] - 122s 122ms/step - loss: 1.0679 - acc: 0.6055 - val_loss: 0.7752 - val_acc: 0.7053


In [25]:
Y_pred = model.predict_generator(validation_generator, num_test_samples // batch_size+1)
y_pred = np.argmax(Y_pred, axis=1)
print('Confusion Matrix')
print(confusion_matrix(validation_generator.classes, y_pred))
print('Classification Report')
target_names = ['1_colony', '2_colonies', '3_colonies', '4_colonies', '5_colonies', '6_colonies', 'outlier']
print(classification_report(validation_generator.classes, y_pred, target_names=target_names))


Confusion Matrix
[[1762 1148  977   18  222   68   46]
 [ 766   52  162    8  201  121  286]
 [1101   36    1    0    0    0    1]
 [ 554   10    3    0    0    0    1]
 [ 282    4    0    0    0    0    2]
 [ 300    4    0    0    0    0    0]
 [ 382    3    2    0    0    0    1]]
Classification Report
              precision    recall  f1-score   support

    1_colony       0.34      0.42      0.38      4241
  2_colonies       0.04      0.03      0.04      1596
  3_colonies       0.00      0.00      0.00      1139
  4_colonies       0.00      0.00      0.00       568
  5_colonies       0.00      0.00      0.00       288
  6_colonies       0.00      0.00      0.00       304
     outlier       0.00      0.00      0.00       388

    accuracy                           0.21      8524
   macro avg       0.06      0.06      0.06      8524
weighted avg       0.18      0.21      0.19      8524



In [15]:
# https://gist.github.com/RyanAkilos/3808c17f79e77c4117de35aa68447045

Y_pred = model.predict_generator(validation_generator, num_test_samples // batch_size+1)
y_pred = np.argmax(Y_pred, axis=1)
print('Confusion Matrix')
print(confusion_matrix(validation_generator.classes, y_pred))
print('Classification Report')
target_names = ['1_colony', '2_colonies', '3_colonies', '4_colonies', '5_colonies', '6_colonies', 'outlier']
print(classification_report(validation_generator.classes, y_pred, target_names=target_names))


Confusion Matrix
[[4148   77    4    1    0    1   10]
 [ 126 1328  128    6    0    3    5]
 [  34  174  805  116    5    5    0]
 [  15   12  125  331   48   37    0]
 [   5    2   11   98   67  105    0]
 [   7    1    5   22   23  212   34]
 [  40    7    1    0    0    0  340]]
Classification Report
              precision    recall  f1-score   support

    1_colony       0.95      0.98      0.96      4241
  2_colonies       0.83      0.83      0.83      1596
  3_colonies       0.75      0.71      0.73      1139
  4_colonies       0.58      0.58      0.58       568
  5_colonies       0.47      0.23      0.31       288
  6_colonies       0.58      0.70      0.64       304
     outlier       0.87      0.88      0.88       388

    accuracy                           0.85      8524
   macro avg       0.72      0.70      0.70      8524
weighted avg       0.84      0.85      0.84      8524



## Balance Classes

In [6]:
model.compile(optimizer=sgd, loss='categorical_crossentropy', metrics=['accuracy'])
balanced_history = model.fit_generator(train_generator,
                                       verbose=1, 
                                       steps_per_epoch=1000, 
                                       epochs=150 , 
                                       validation_data=validation_generator,
                                      validation_steps=300,
                                      class_weight=class_weights)
model.save('balanced_classes.hdf5')

W0822 18:06:15.504884 13624 deprecation_wrapper.py:119] From C:\Users\acsch\Anaconda3\envs\Tensorflow-GPU\lib\site-packages\keras\optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W0822 18:06:15.572340 13624 deprecation.py:323] From C:\Users\acsch\Anaconda3\envs\Tensorflow-GPU\lib\site-packages\tensorflow\python\ops\math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Epoch 1/150
1000/1000 [==============================] - 170s 170ms/step - loss: 1.2578 - acc: 0.5642 - val_loss: 0.9296 - val_acc: 0.6526
Epoch 2/150
1000/1000 [==============================] - 119s 119ms/step - loss: 0.9432 - acc: 0.6464 - val_loss: 0.7786 - val_acc: 0.7135
Epoch 3/150
1000/1000 [==============================] - 119s 119ms/step - loss: 0.8306 - acc: 0.6833 - val_loss: 0.7435 - val_acc: 0.7183
Epoch 4/150
1000/1000 [==============================] - 120s 120ms/step - loss: 0.7664 - acc: 0.7074 - val_loss: 0.6653 - val_acc: 0.7457
Epoch 5/150
1000/1000 [==============================] - 120s 120ms/step - loss: 0.7080 - acc: 0.7277 - val_loss: 0.7551 - val_acc: 0.7076
Epoch 6/150
1000/1000 [==============================] - 120s 120ms/step - loss: 0.6670 - acc: 0.7421 - val_loss: 0.6814 - val_acc: 0.7409
Epoch 7/150
1000/1000 [==============================] - 120s 120ms/step - loss: 0.6365 - acc: 0.7516 - val_loss: 0.6727 - val_acc: 0.7340
Epoch 8/150
1000/1000 [====

1000/1000 [==============================] - 119s 119ms/step - loss: 0.3438 - acc: 0.8679 - val_loss: 0.3621 - val_acc: 0.8697
Epoch 118/150
1000/1000 [==============================] - 119s 119ms/step - loss: 0.3382 - acc: 0.8705 - val_loss: 0.3776 - val_acc: 0.8612
Epoch 119/150
1000/1000 [==============================] - 120s 120ms/step - loss: 0.3386 - acc: 0.8691 - val_loss: 0.4373 - val_acc: 0.8474
Epoch 120/150
1000/1000 [==============================] - 120s 120ms/step - loss: 0.3360 - acc: 0.8694 - val_loss: 0.4420 - val_acc: 0.8357
Epoch 121/150
1000/1000 [==============================] - 120s 120ms/step - loss: 0.3374 - acc: 0.8713 - val_loss: 0.4796 - val_acc: 0.8228
Epoch 122/150
1000/1000 [==============================] - 119s 119ms/step - loss: 0.3315 - acc: 0.8721 - val_loss: 0.3720 - val_acc: 0.8698
Epoch 123/150
1000/1000 [==============================] - 119s 119ms/step - loss: 0.3390 - acc: 0.8709 - val_loss: 0.3509 - val_acc: 0.8738
Epoch 124/150
1000/1000 [==

In [7]:
model = keras.models.load_model('balanced_classes.hdf5')
Y_pred = model.predict_generator(validation_generator, num_test_samples // batch_size+1)
y_pred = np.argmax(Y_pred, axis=1)
print('Confusion Matrix')
print(confusion_matrix(validation_generator.classes, y_pred))
print('Classification Report')
target_names = ['1_colony', '2_colonies', '3_colonies', '4_colonies', '5_colonies', '6_colonies', 'outlier']
print(classification_report(validation_generator.classes, y_pred, target_names=target_names))


Confusion Matrix
[[4187   46    2    0    0    0    6]
 [ 141 1348   99    5    0    2    1]
 [  40  194  840   59    4    1    1]
 [  14   15  165  322   36   16    0]
 [   5    2   24  114   82   61    0]
 [   9    1    4   32   47  210    1]
 [  25    9    1    0    0    0  353]]
Classification Report
              precision    recall  f1-score   support

    1_colony       0.95      0.99      0.97      4241
  2_colonies       0.83      0.84      0.84      1596
  3_colonies       0.74      0.74      0.74      1139
  4_colonies       0.61      0.57      0.59       568
  5_colonies       0.49      0.28      0.36       288
  6_colonies       0.72      0.69      0.71       304
     outlier       0.98      0.91      0.94       388

    accuracy                           0.86      8524
   macro avg       0.76      0.72      0.73      8524
weighted avg       0.85      0.86      0.86      8524



## Imbalanced Classes

In [22]:
model.compile(optimizer='sgd', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit_generator(train_generator,
                    verbose=1, 
                    steps_per_epoch=1000, 
                    epochs=1, 
                    validation_data=validation_generator,
                   validation_steps=300,
                   class_weight=class_weights)

Epoch 1/1
1000/1000 [==============================] - 120s 120ms/step - loss: 0.9240 - acc: 0.6460 - val_loss: 0.8217 - val_acc: 0.7030


In [23]:
Y_pred = model.predict_generator(validation_generator, num_test_samples // batch_size+1)
y_pred = np.argmax(Y_pred, axis=1)
print('Confusion Matrix')
print(confusion_matrix(validation_generator.classes, y_pred))
print('Classification Report')
target_names = ['1_colony', '2_colonies', '3_colonies', '4_colonies', '5_colonies', '6_colonies', 'outlier']
print(classification_report(validation_generator.classes, y_pred, target_names=target_names))


Confusion Matrix
[[2740 1107  328   27    0   14   25]
 [ 221  446  550   94   13  257   15]
 [ 760   41   21    3    2   45  267]
 [ 552   15    0    0    0    0    1]
 [ 277   10    0    0    0    0    1]
 [ 290   13    0    0    0    0    1]
 [ 378   10    0    0    0    0    0]]
Classification Report
              precision    recall  f1-score   support

    1_colony       0.53      0.65      0.58      4241
  2_colonies       0.27      0.28      0.28      1596
  3_colonies       0.02      0.02      0.02      1139
  4_colonies       0.00      0.00      0.00       568
  5_colonies       0.00      0.00      0.00       288
  6_colonies       0.00      0.00      0.00       304
     outlier       0.00      0.00      0.00       388

    accuracy                           0.38      8524
   macro avg       0.12      0.13      0.13      8524
weighted avg       0.32      0.38      0.34      8524

